In [2]:
import json
from collections import OrderedDict, defaultdict

In [19]:
def convert_sent_dict(doc_key, sentence):
    global predicate_tag_count
    global arg_count
    speakers = [['-' for _ in sentence]]
    sentences = [[line[1] for line in sentence]]
    constituents = [[]]
    clusters = []
    ner = [[]]
    srl = [[]]
    predicate_idx = 0
    for idx in range(len(sentence)):
        if sentence[idx][12] == 'Y': # is predicate
            no_arg = True
            predicate_tag_count[sentence[idx][4][:2]] += 1
            for jdx in range(len(sentence)):
                if sentence[jdx][14+predicate_idx] != '_':
                    no_arg = False
                    srl[0].append([idx, jdx, jdx, sentence[jdx][14+predicate_idx]])
                    arg_count[sentence[jdx][14+predicate_idx]] += 1
            predicate_idx += 1
            if no_arg: # in case of no argument
                srl[0].append([idx, idx, idx, '_'])
    return {
        'speakers':speakers,
        'doc_key':doc_key,
        'sentences':sentences,
        'srl':srl,
        'constituents':constituents,
        'clusters':clusters,
        'ner':ner
    }



def conll09_to_json(dataset_path, output_path):
    with open(dataset_path, 'r') as f:
        data = f.readlines()
    
    sentences = []
    sent = []
    for line in data:
        if len(line.strip())>0:
            sent.append(line.strip().split('\t'))
        else:
            if len(sent)>0:
                sentences.append(sent)
                sent = []
    
    if len(sent)>0:
        sentences.append(sent)

    with open(output_path, 'w') as f:
        for idx in range(len(sentences)):
            json_data = convert_sent_dict('S'+str(idx), sentences[idx])
            f.write(json.dumps(json_data)+'\n')



In [20]:
dataset_path = "/home/shtoshni/Research/srl/data/conll-2009/conll-2009-p2/data/CoNLL2009-ST-English/CoNLL2009-ST-English-train.txt"
output_path = "/tmp/output.json"

global predicate_tag_count
global arg_count

predicate_tag_count = defaultdict(int)
arg_count = defaultdict(int)

In [21]:
conll09_to_json(dataset_path, output_path)

In [24]:
print(predicate_tag_count)
print()
print(sorted(list(arg_count.items()),  key= lambda x: x[1], reverse=True))

defaultdict(<class 'int'>, {'NN': 89780, 'VB': 89193, 'IN': 15, 'RB': 5, 'PR': 12, 'CD': 7, 'TO': 1, 'JJ': 1})

[('A1', 146548), ('A0', 99388), ('A2', 46741), ('AM-TMP', 23347), ('AM-MNR', 11837), ('AM-LOC', 10387), ('A3', 9431), ('AM-MOD', 9033), ('AM-ADV', 8541), ('AM-DIS', 4825), ('R-A0', 4039), ('AM-NEG', 3653), ('A4', 3064), ('C-A1', 2754), ('R-A1', 2307), ('AM-PNC', 2233), ('AM-EXT', 1309), ('AM-CAU', 1220), ('AM-DIR', 1146), ('R-AM-TMP', 704), ('R-A2', 286), ('R-AM-LOC', 212), ('R-AM-MNR', 141), ('A5', 82), ('AM-PRD', 69), ('C-A0', 64), ('C-A2', 61), ('R-AM-CAU', 41), ('C-A3', 35), ('R-A3', 28), ('C-AM-MNR', 20), ('C-AM-ADV', 20), ('AM-REC', 14), ('AA', 13), ('R-AM-PNC', 12), ('C-AM-EXT', 11), ('C-A4', 11), ('C-AM-TMP', 10), ('AM', 9), ('C-AM-LOC', 9), ('R-A4', 7), ('C-AM-CAU', 6), ('R-AM-ADV', 5), ('C-AM-PNC', 5), ('C-AM-DIR', 5), ('R-AM-EXT', 4), ('C-AM-DIS', 3), ('AM-PRT', 2), ('AM-TM', 2), ('R-AA', 2), ('C-AM-NEG', 1), ('C-R-AM-TMP', 1), ('R-AM-DIR', 1)]
